In [ ]:
import cantera as ct
import cantera.ck2cti as ck2cti
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## Converting from Chemkin

In [ ]:
# command line equivalent:
# python -m cantera.ck2cti --input=mech.txt --thermo=thermo.txt --transport=tran.txt --permissive --out=mech.cti
parser = ck2cti.Parser()
#parser.convertMech('mech.txt', 'thermo.txt', 'tran.txt', outName='mech.cti')
#parser.convertMech('mech_fixed.txt', 'thermo.txt', 'tran.txt', outName='mech.cti')
#parser.convertMech('mech_fixed.txt', 'thermo_fix.txt', 'tran.txt', outName='mech.cti')
parser.convertMech('mech_fixed.txt', 'thermo_fixed.txt', 'tran.txt', permissive=True, outName='mech.cti')

## Thermo Data

In [ ]:
gas = ct.Solution('mech.cti')

In [ ]:
sp = gas.species('H2CNO')
T = np.linspace(600, 2000, 200)
f,ax = plt.subplots(1,3, figsize=(8,3.5))

def plot_thermo(thermo):
    h = [thermo.h(tt)/(ct.gas_constant * tt) for tt in T]
    cp = [thermo.cp(tt)/ct.gas_constant for tt in T]
    s = [thermo.s(tt)/ct.gas_constant for tt in T]
    ax[0].plot(T,cp)
    ax[0].set_title('$c_p/R$')
    ax[1].plot(T,h)
    ax[1].set_title('$h/RT$')
    ax[2].plot(T,s)
    ax[2].set_title('$s/R$')
    f.tight_layout()
plot_thermo(sp.thermo)

In [ ]:
c0 = sp.thermo.coeffs
c0

In [ ]:
c0[0] = 1200
test = ct.NasaPoly2(sp.thermo.min_temp, sp.thermo.max_temp, sp.thermo.reference_pressure, c0)
plot_thermo(test)

In [ ]:
c0[0] = 1000
test = ct.NasaPoly2(sp.thermo.min_temp, sp.thermo.max_temp, sp.thermo.reference_pressure, c0)
plot_thermo(test)

## Reaction Rates

In [ ]:
gas.TPX = 300, 101325, 'CH4:1.0, O2:0.1'
gas.equilibrate('TP')

In [ ]:
f,ax = plt.subplots(1,1)
ax.semilogy(gas.forward_rate_constants, 'b.')
ax.semilogy(gas.reverse_rate_constants, 'r.')
ax.set_ylim(ymin=1e-30)
f.tight_layout()

In [ ]:
kr = gas.reverse_rate_constants
for i,k in enumerate(kr):
    if k > 1e20:
        print(i,k, gas.reaction_equation(i))

In [ ]:
gri = ct.Solution('gri30.cti')
for i,r in enumerate(gri.reactions()):
    if 'CH3' in r and 'H2' in r and 'CH' in r:
        print(i, r)

In [ ]:
gasN = ct.SolutionArray(gas, shape=200)
griN = ct.SolutionArray(gri, shape=200)
T = np.linspace(300, 3000, 200)
gasN.TPY = T, 101325, 'N2:1.0'
griN.TPY = T, 101325, 'N2:1.0'

f,ax = plt.subplots(1,1, figsize=(5,4))
ax.semilogy(1000/T, gasN.forward_rate_constants[:,295], label='mech, forward')
ax.semilogy(1000/T, gasN.reverse_rate_constants[:,295], label='mech, reverse')
ax.semilogy(1000/T, griN.forward_rate_constants[:,288], '--', label='GRI3.0, forward')
ax.semilogy(1000/T, griN.reverse_rate_constants[:,288], '--', label='GRI3.0, reverse')
ax.legend(loc='best')
ax.grid(True)
f.tight_layout()